# Automating Azure SQL Diagnostic notebook execution

In Azure Automation, create a Runbook and choose PowerShell as the Runbook type.

Create a service principal as per [Create Azure AD users using service principals - Azure SQL Database | Microsoft Docs](https://docs.microsoft.com/en-us/azure/azure-sql/database/authentication-aad-service-principal-tutorial#create-the-service-principal-user-in-azure-sql-database). The name of the Azure Automation is the user name that you will have to create in the service database. 

Azure Automation Runbook is using Managed Identity for execution. More info about Managed Identity is [here](https://docs.microsoft.com/en-us/powershell/module/sqlserver/invoke-sqlcmd?view=sqlserver-ps&source=docs#example-11--connect-to-azure-sql-database--or-managed-instance--using-an-access-token)

Create an Azure Key Vault Secrets to access the blob storage where the output notebook will be stored. Setup Access Policy as per [Assign an Azure Key Vault access policy (Portal) | Microsoft Docs](https://docs.microsoft.com/en-us/azure/key-vault/general/assign-access-policy-portal) to link the Azure Automation account to the blob storage.

Below is the Azure Automation Runbook definition.

In [ ]:
param (
      [Parameter(Mandatory=$true)][String]$Title      
   )
       
    if($Title -match "\[([^\]]+)\]\[([^\]]+)\]")
    {
        # Server and database
        $ServerName = $matches[1]
        $DatabaseName = $matches[2]

        Import-Module SQLServer
        Import-Module Az.Accounts -MinimumVersion 2.2.0

        # Logging in 
        $ac = Connect-AzAccount -Identity

        # Set the file name
        $inputNB = "DBDiagnostic.ipynb"
        $outputNB = "DBDiagnostic_$(get-date -f yyyy-MM-dd).ipynb"

        # Downloading the notebook that we want to run and store it in inputNB     
        $sourceNB = "https://raw.githubusercontent.com/MsSQLGirl/jubilant-data-wizards/main/Simple%20Demo/SQL%20Notebooks/DBDiagnostics.ipynb"
        Invoke-WebRequest $sourceNB -OutFile $inputNB

        # Getting access token for this app to connect to Azure SQL         
        $sqlAccessToken = (Get-AzAccessToken -ResourceUrl https://database.windows.net).Token

        # Now that we have the token, we use it to connect to an Azure SQL DB and run the notebook
        # inputNB and save the executed notebook as outputNB        
        $result = Invoke-SqlNotebook -ServerInstance $ServerName -Database $DatabaseName -AccessToken $sqlAccessToken `
                        -InputFile $inputNB -OutputFile $outputNB  

        # Get the SAS token for Azure Storage               
        $secret = Get-AzKeyVaultSecret -VaultName "myAKV" -Name "mySASKey"
        $ssPtr = [System.Runtime.InteropServices.Marshal]::SecureStringToBSTR($secret.SecretValue)
        try {
            $storageAccountKey = [System.Runtime.InteropServices.Marshal]::PtrToStringBSTR($ssPtr)
        } finally {
            [System.Runtime.InteropServices.Marshal]::ZeroFreeBSTR($ssPtr)
        }

        # Write to Storage Account
        $storageAccount = "mystorageaccount"        
        $blob = "myblob"

        # Saving to blob and return the Uri
        $result = New-AzStorageContext -StorageAccountName $storageAccount -StorageAccountKey $storageAccountKey `
            | Set-AzStorageBlobContent  -Container $blob  -File $outputNB -Blob $outputNB -Force

        $blobUri = $result.ICloudBlob.Uri.AbsoluteUri

        # Return the URI of the output notebook
        $blobUri
    }

